# Cleaned Up Dataset


In [65]:
!pip install wbgapi


In [66]:
import wbgapi as wb
import pandas as pd
from functools import reduce

# Define the World Bank indicators you want
indicators = {
    'GDP_per_capita': 'NY.GDP.PCAP.KD',
    'GDP_growth': 'NY.GDP.MKTP.KD.ZG',
    'Primary_enrollment': 'SE.PRM.ENRR',
    'Secondary_enrollment': 'SE.SEC.ENRR',
    'Tertiary_enrollment': 'SE.TER.ENRR',
    'Education_expenditure': 'SE.XPD.TOTL.GD.ZS',
    'Population': 'SP.POP.TOTL'
}

years = list(range(2000, 2023))
all_data = []

# Pull each indicator separately
for col_name, indicator_code in indicators.items():
    print(f"📦 Downloading {col_name}...")
    temp = wb.data.DataFrame(indicator_code, time=years, labels=False, columns='series', index=['economy', 'time'])
    temp.reset_index(inplace=True)
    temp.rename(columns={'economy': 'country', 'time': 'year', indicator_code: col_name}, inplace=True)
    
    # Only keep what we need for clean merging
    temp = temp[['country', 'year', col_name]]
    all_data.append(temp)

# Merge all indicator tables on country + year
merged_df = reduce(lambda left, right: pd.merge(left, right, on=['country', 'year'], how='outer'), all_data)

# Save to CSV
merged_df.to_csv("wdi_cleaned_data.csv", index=False)
print("✅ All indicators saved to 'wdi_cleaned_data.csv'")






📦 Downloading GDP_per_capita...
📦 Downloading GDP_growth...
📦 Downloading Primary_enrollment...
📦 Downloading Secondary_enrollment...
📦 Downloading Tertiary_enrollment...
📦 Downloading Education_expenditure...
📦 Downloading Population...
✅ All indicators saved to 'wdi_cleaned_data.csv'


In [67]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("wdi_project.db")

# Query to fetch the first 10 rows of the wdi_raw table
query = "SELECT * FROM wdi_raw LIMIT 10;"
wdi_raw_sample = pd.read_sql_query(query, conn)

# Display the result
print(wdi_raw_sample)

  country  year  GDP_per_capita  GDP_growth  Primary_enrollment  \
0     ABW  2000    30199.661277    7.622921          108.326508   
1     ABW  2001    31169.796914    4.182002          110.920418   
2     ABW  2002    30662.321715   -0.944953          115.317001   
3     ABW  2003    30651.945756    1.110505          113.902641   
4     ABW  2004    32192.791947    7.293728          116.187538   
5     ABW  2005    31249.278037   -0.383138          116.397911   
6     ABW  2006    31038.889867    1.127411          117.467499   
7     ABW  2007    31759.823202    3.089544          119.884132   
8     ABW  2008    32097.040224    1.835755          118.331169   
9     ABW  2009    28157.148989  -11.677741          120.840927   

   Secondary_enrollment  Tertiary_enrollment  Education_expenditure  \
0             93.762329            27.973761                4.71468   
1             95.286102            29.228010                4.79898   
2            100.267097            28.924419     

In [71]:
import sqlite3
import pandas as pd
import numpy as np
import wbgapi as wb

# Step 1: Connect to DB
conn = sqlite3.connect("wdi_project.db")

# Step 2: Load full raw table
df = pd.read_sql_query("SELECT * FROM wdi_raw", conn)

# Step 3: Get only real countries (no aggregates)
valid_countries_df = wb.economy.DataFrame(skipAggs=True)
valid_countries = valid_countries_df.index.tolist()

# Step 4: Filter to real countries
df = df[df['country'].isin(valid_countries)]

# Step 5: Get top 15 countries by population in 2022
pop_2022 = df[df['year'] == 2022][['country', 'Population']]
top15_countries = pop_2022.sort_values(by='Population', ascending=False).head(15)['country'].tolist()

# Step 6: Filter full data to those 15 countries
df_top15 = df[df['country'].isin(top15_countries)]

# Step 7: Sort by country and year
df_top15 = df_top15.sort_values(by=['country', 'year'])

# Step 8: Compute YoY % change for selected indicators
indicators = ['GDP_per_capita', 'Primary_enrollment', 'Secondary_enrollment', 'Tertiary_enrollment']

for col in indicators:
    yoy_col = f'YoY_{col}'
    df_top15[yoy_col] = df_top15.groupby('country')[col].pct_change() * 100
    df_top15[yoy_col] = df_top15[yoy_col].round(2)

# Step 9: Save to CSV
df_top15.to_csv("top15_countries_with_yoy.csv", index=False)
print("✅ File saved as 'top15_countries_with_yoy.csv' with year-over-year % changes")

# Step 10: Close DB connection
conn.close()


✅ File saved as 'top15_countries_with_yoy.csv' with year-over-year % changes


/var/folders/6l/xfqxt4zj5rvdhhf45x8rl18h0000gn/T/ipykernel_85432/125329031.py:34: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_top15[yoy_col] = df_top15.groupby('country')[col].pct_change() * 100
/var/folders/6l/xfqxt4zj5rvdhhf45x8rl18h0000gn/T/ipykernel_85432/125329031.py:34: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_top15[yoy_col] = df_top15.groupby('country')[col].pct_change() * 100
/var/folders/6l/xfqxt4zj5rvdhhf45x8rl18h0000gn/T/ipykernel_85432/125329031.py:34: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be r